In [1]:
import re

parser_group = re.compile(r"(\d+) units each with (\d+) hit points (|\((?:.+)\) )with an attack that does (\d+) (\w+) damage at initiative (\d+)")
parser_team = re.compile(r"(?:Immune System|Infection):")
parser_weakness = re.compile(r"weak to (\w+(?:, \w+)*)")
parser_immunities = re.compile(r"immune to (\w+(?:, \w+)*)")

class Group:
    def __init__(self, group_id, team, units, hp, weaknesses, immunities, damage, damage_type, initiative):
        self.group_id = group_id
        self.team = team
        self.units = units
        self.hp = hp
        self.weaknesses = weaknesses
        self.immunities = immunities
        self.damage = damage
        self.damage_type = damage_type
        self.initiative = initiative
        
    def effective_power(self):
        return self.units * self.damage
    
    def targeting_order_key(self):
        return (self.effective_power(), self.initiative)
    
    def attacking_order_key(self):
        return self.initiative
    
    def __str__(self):
        result = "{} units each with {} hit points".format(self.units, self.hp)
        if self.weaknesses or self.immunities:
            result += " ("
        if self.weaknesses:
            result += "weak to {}".format(", ".join(self.weaknesses))
            if self.immunities:
                result += "; "
        if self.immunities:
            result += "immune to {}".format(", ".join(self.immunities))
        if self.weaknesses or self.immunities:
            result += ")"
        result += " with an attack that does {} {} damage at initiative {}".format(self.damage, self.damage_type, self.initiative)
        return result

def get_input():
    groups = {}
    with open("Input/24.txt") as file:
        team = -1
        for group_id, line in enumerate(file):
            if parser_team.match(line):
                team += 1
                continue
            m = parser_group.match(line)
            if m:
                units, hp, stats, damage, damage_type, initiative = m.groups()
                units, hp, damage, initiative = int(units), int(hp), int(damage), int(initiative)

                mw = parser_weakness.search(stats)
                if mw:
                    weaknesses = mw.group(1).split(", ")
                else:
                    weaknesses = []

                mi = parser_immunities.search(stats)
                if mi:
                    immunities = mi.group(1).split(", ")
                else:
                    immunities = []

                group = Group(group_id, team, units, hp, weaknesses, immunities, damage, damage_type, initiative)
                groups[group_id] = group
    return groups

def print_groups(groups):
    print("Immune System:")
    for g in [g for g in groups.values() if g.team == 0]:
        print(g)
    print()
    print("Infection:")
    for g in [g for g in groups.values() if g.team == 1]:
        print(g)

In [2]:
groups = get_input()
print_groups(groups)

Immune System:
4445 units each with 10125 hit points (immune to radiation) with an attack that does 20 cold damage at initiative 16
722 units each with 9484 hit points with an attack that does 130 bludgeoning damage at initiative 6
1767 units each with 5757 hit points (weak to fire, radiation) with an attack that does 27 radiation damage at initiative 4
1472 units each with 7155 hit points (weak to slashing, bludgeoning) with an attack that does 42 radiation damage at initiative 20
2610 units each with 5083 hit points (weak to slashing, fire) with an attack that does 14 fire damage at initiative 17
442 units each with 1918 hit points with an attack that does 35 fire damage at initiative 8
2593 units each with 1755 hit points (immune to bludgeoning, radiation, fire) with an attack that does 6 slashing damage at initiative 13
6111 units each with 1395 hit points (weak to bludgeoning; immune to radiation, fire) with an attack that does 1 slashing damage at initiative 14
231 units each wit

In [3]:
def calc_damage(attacker, defender):
    if attacker.team == defender.team:
        return 0
    if attacker.damage_type in defender.immunities:
        return 0
    if attacker.damage_type in defender.weaknesses:
        return 2 * attacker.effective_power()
    return attacker.effective_power()

def targeting_priority(attacker, defender): 
    return (calc_damage(attacker, defender), defender.effective_power(), defender.initiative)

In [4]:
def targeting(groups):
    targets = {}
    targeting_order = [g.group_id for g in sorted(groups.values(), key = Group.targeting_order_key, reverse = True)]
    for attacker_id in targeting_order:
        attacker = groups[attacker_id]
        target = max((g for g in groups.values() if g.team != attacker.team and g.group_id not in targets.values()), key = lambda g : targeting_priority(attacker, g), default = None)
        if not target or calc_damage(attacker, target) == 0:
            continue
        targets[attacker_id] = target.group_id
    return targets

In [5]:
def attacking(groups, targets):
    state_changed = False
    attack_order = [g.group_id for g in sorted(groups.values(), key = Group.attacking_order_key, reverse = True)]
    for attacker_id in attack_order:
        if attacker_id not in targets or attacker_id not in groups:
            continue
        target_id = targets[attacker_id]
        attacker = groups[attacker_id]
        target = groups[target_id]
        units_fallen = calc_damage(attacker, target) // target.hp
        if units_fallen == 0:
            continue
        target.units -= units_fallen
        state_changed = True
        if target.units <= 0:
            del groups[target_id]
    return state_changed

In [6]:
def turn(groups):
    if not groups:
        # No groups remaining
        return False
    if len(set(g.team for g in groups.values())) == 1:
        # A team has eliminated the other
        return False
    targets = targeting(groups)
    if not targets:
        # Nobody can attack anyone
        return False
    if not attacking(groups, targets):
        # No unit killed during this turn
        return False
    return True

In [7]:
def play(groups):
    while turn(groups):
        pass

In [8]:
groups = get_input()
play(groups)
part1 = sum(g.units for g in groups.values())
print("Part 1: {}".format(part1))

Part 1: 21891


In [14]:
low = 0
high = 8192

while low <= high:
    mid = (low + high) // 2
    groups = get_input()
    for g in groups.values():
        if g.team == 0:
            g.damage += mid
    play(groups)
    if all(g.team == 0 for g in groups.values()):
        # Immune system won: the boost is at least high enough
        high = mid - 1
    else:
        low = mid + 1

groups = get_input()
for g in groups.values():
    if g.team == 0:
        g.damage += low
play(groups)
part2 = sum(g.units for g in groups.values())
print("Part 2: {} (requires {} boost)".format(part2, low))

Part 2: 7058 (requires 82 boost)
